Gorth16 in Klefki
====================

## Bilinear groups

We will work over bilinear groups $(p,\mathbb{G}_1,\mathbb{G}_2,\mathbb{G}_T,e,g,h)$ with the following properties:

* $\mathbb{G}_1, \mathbb{G}_2, mathbb{G}_T$ are groups of prime order p

* The pairing $e$: $\mathbb{G}_1x\mathbb{G}_2 \rightarrow \mathbb{G}_T$ is a binlinear map

* $g$ is a generator for $\mathbb{G}_1$, $h$ is a generator for $\mathbb{G}_2$ and $e(g, h)$ is a generator for $\mathbb{G}_T$

* There are efficient algorithms for computing group operations, evaluating the bilinearmap, deciding membership of the groups, deciding equality of group elements andsampling generators of the groups. We refer to these as the generic group operations.

In [15]:
from klefki.curves.barreto_naehrig.bn128 import BN128FP as FP, BN128FP2 as FP2, BN128FP12 as FP12, ECGBN128 as ECG
from klefki.algebra.utils import randfield

In [16]:
g = ECG.G1
h = ECG.G2

We write $[a]_1$ for $g^a$, $[b]_2$ for $h^b$, and $[c]_T$ for $e(g, h)^c$. A vector of group elements will be represented as $[\mathbf{a}]_i$. We defined dot product as $[\mathbf{a}]_1 \circ [\mathbf{b}]_2 = [\mathbf{a}\circ \mathbf{b}]_T$

## NIZK

let $R$ be a relation generator that given a security parameter $λ$ in unary returns a polynomial time decidable binary relation $R$. For pairs $(\phi, \omega) \in R$.

For pairs $(\phi, \omega) \ in R$, we call $\phi$ the statement and $\omega$ the witness. We defined $R_{\lambda}$  to be the set of possible relations $R $the relation gen-erator may output given $1^λ$. The relation generator may also output someside information, an auxiliary input $z$, which will be given to the adversary.

An efficient prover publicly verifiable non-interactive argument for $R$ is a quadruple of probabilistic polynomial algorithms $(Setup,Prove,Vfy,Sim)$ such that

* $(\sigma,\tau) \leftarrow Setup(R)$: The setup produces a common reference stringσand a simulationtrapdoorτfor the relationR

* $\pi \leftarrow Prov(R, \sigma, \phi, \omega)$:The prover algorithm takes as input a common reference string σ and $(φ,w)∈R$ and returns an argument π.

* $0/1 \leftarrow Vfy(R, \sigma, \phi, \pi)$: The  verification  algorithm  takes  as  input  a  common  referencestring σ, a statement φ and an argumentπand returns 0 (reject) or 1 (accept).

* $\pi \leftarrow Sim(R, \tau, \phi)$: The simulator takes as input a simulation trapdoor and statementφand returns an argument π.

## QAP

Given $n$ equations we pick arbitrary distinct $r_1,\cdots,r_n \in \mathbb{F}$ and define
$$
t(x) = \prod_{q=1}^n (x - r_q)
$$

$$
\sum_{i=0}^m a_i u_i(X) \circ \sum_{i=0}^m a_i v_i(X) = \sum_{i=0}^m a_i w_i(X) + h(X)t(X)
$$

 we will be working with quadratic arithmetic programsRthat have thefollowing description
 
 $$
 R = (p, \mathbb{G}_1,\mathbb{G}_2, \mathbb{G}_T, e, g, h, l, \{u_i(X), v_i(X), w_i(X)\}_{i=0}^n, t(X))
 $$

with $|p|=\lambda$. The realation defineds a field $\mathbb{Z}_p$ and a language of statement $(a_i,\cdots, a_l) \in \mathbb{Z}_p^l$ and witness $(a_{l+1}, \cdots, a_m) \in \mathbb{Z}_p^{m-l}$ such that with $a_0=1$

In [17]:
from collections import namedtuple

RationalGenerator = namedtuple("RelationGenerator", ["F", "G1", "G2", "Gt", "e", "g", "h", "l", "U", "V", "W", "T"])

In [18]:
from klefki.zkp.r1cs import R1CS
from klefki.zkp.qap import QAP


In [19]:
@R1CS.r1cs(FP)
def f(x, k, c):
    y = x + c + k
    return y ** 3


In [20]:
qap = QAP(f.A, f.B, f.C)
U, V, W, T = qap.qap
a = f.witness(FP(89), FP(8), FP(8))

In [21]:
R = RationalGenerator(FP, ECG, ECG, ECG, ECG.e, ECG.G1, ECG.G2, 3, U, V, W, T)

## $(\sigma,\tau) \leftarrow Setup(R)$:

Pick $\alpha, \beta, \gamma, \delta, x \leftarrow \mathbb{Z}_P^*$, Define $\tau = (\alpha, \beta, \gamma, \delta, x)$ and compute $\sigma = ([\mathbf{\sigma_1}]_1, [\mathbf{\sigma_2}]_2$ where

$$
(\mathbf{\sigma}_1, \mathbf{\sigma}_2) = \left( \left(\begin{split}
\alpha, \beta, \delta, \{x^i\}_{i=0}^{n-1},
\left\{\frac{\beta u_i(x) + \alpha v_i (x) + w_i (x)}{\gamma} \right\}_{i=0}^l \\
\left\{ \frac{\beta u_i (x) + \alpha v_i(x) + w_i(x)}{\delta} \right\}_{i=l+1}^m,
\left\{\frac{x^i t(x)}{\delta} \right \}_{i=0}^{n-2}
\end{split}\right) , \left(\beta, \gamma, \delta, \{x^i\}^{n-1}_{i=0} \right)
\right)
$$

In [22]:
from typing import Iterable, Tuple, Type
from operator import add
from functools import reduce

def setup(R, n) -> Tuple[Iterable, Tuple[R.G1, R.G2]]:
    tau = alpha, beta, delta, gamma, x = \
        randfield(R.F), randfield(R.F), randfield(R.F), randfield(R.F), randfield(R.F)
    n = R.U[0].degree
    sigma_1 = [alpha, beta, delta] + \
              [x ** i for i in range(0, n-1)] + \
              [(R.U[i](x) * beta + R.V[i](x) * alpha + R.W[i](x)) / gamma  for i in range(0, R.l)] + \
              [(R.U[i](x) * beta + R.V[i](x) * alpha + R.W[i](x)) / delta  for i in range(R.l + 1, len(R.U))] + \
              [(x ** i * R.T(x)) / delta for i in range(0, n-2)]
    
    sigma_2 = [beta, gamma, delta] + \
              [x**i for i in (0, n-1)]
    
    sigma = ([R.g @ s for s in sigma_1], [R.h @ s for s in sigma_2])
    return tau, sigma

In [23]:
tau, sigma = setup(R, 3)

## $\pi \leftarrow Prov(R, \tau, \sigma, a_1, \cdots, a_m)$

Pick $r, s \leftarrow \mathbb{Z}_p$ and compute $\pi = ([A]_1, [C]_1, [B]_2$ where

$$
A = \alpha + \sum_{i=0}^m a_i u_i(x) + r\delta
$$

$$
B = \beta + \sum_{i=0}^m a_i v_i(x) + s\delta
$$

$$
C = \frac{\sum_{i=l+1}^m a_i(\beta u_i(x) + \alpha v_i(x) + w_i(x)) + h(x)t(x)}{\delta} + As + Br - rs\delta
$$

In [24]:
H = qap.H(a)

In [25]:
def prov(R, tau, sigma, a) -> Tuple[R.G1, R.G1, R.G2]:
    r, s = randfield(R.F), randfield(R.F)
    alpha, beta, delta, gamma, x = tau
    m = len(a)
    A = alpha + reduce(add, [a[i] * (R.U[i](x)) for i in range(0, m)]) + r * delta
    B = beta + reduce(add, [a[i] * (R.V[i](x)) for i in range(0, m)]) + s * delta
    C = (reduce(add, [a[i]*(beta * R.U[i](x) + alpha * R.V[i](x) + R.W[i](x)) for i in range(R.l + 1, m)]) + H(x) * R.T(x)) / ~delta \
        + (A * s + B * r - r * s * delta)
    return (R.g @ A, R.g @ C, R.h @ B)
    
    

    

In [26]:
pi = prov(R, tau, sigma, a)
pi

(ECGBN128::(BN128FP::18516648705834430349939177579536019654324296679099543941908707472729509911126, BN128FP::17698670626348665920227842489245758548471684407485634495490912829277994864858),
 ECGBN128::(BN128FP::8488791507382916376996280129956285730041643084428968020606257167760402216404, BN128FP::4082680017939613116097175823429222055736854009148188310405390025884468064986),
 ECGBN128::(BN128FP2::[BN128FP::4633278075369866772831604094151143656179576404021857223005441199765708572437, BN128FP::6462225141696652563619115122425856136710130657871571271798221292817136010061], BN128FP2::[BN128FP::1694515370192953786638105026398607108129046797030149208530707508305574614357, BN128FP::4110349820884174906704555092802266024988418670442760401126407706931213959511]))

## $0/1 \leftarrow Vfy(R, \tau, a_1, \cdots, a_l, \pi)$

Parse $\pi = ([A]_1, [C]_1, [B]_2 \in G_1^2 x G_2$. Accept the proof iff:

$$
[A]_1 \circ [B]_2 = [\alpha]_1 \circ [\beta]_2 + \sum_{i=0}^l a_i \left[ \frac{\beta u_i(x) + \alpha v_i(x) + w_i(x)}{\gamma} \right]_1 \circ [\gamma]_2 + [C]_1 \circ [\delta]_2
$$

In [27]:
def vfy(R, tau, a, pi):
    alpha, beta, delta, gamma, x = tau
    A_1, C_1, B_2 = pi
    lhs = R.e(A_1, B_2)
    rhs = R.e(R.g @ alpha, R.h @ beta) + \
        reduce(add, 
            [
                R.e(
                    R.g @ ((beta * R.U[i](x) + alpha * R.V[i](x) + R.W[i](x)) / gamma),
                    R.h @ gamma
                ) * a[i]
                for i in range(0, R.l)
            ]
        ) + \
        R.e(C_1, R.h @ delta)
    return lhs == rhs

In [28]:
vfy(R, tau, a[:R.l], pi)

False